In [ ]:
!git clone https://github.com/ntienhuy/MultiChannel
# !npx degit ntienhuy/MultiChannel/tree/master/data/VS/Data_not_token/Fold_1 -f

Cloning into 'MultiChannel'...
remote: Enumerating objects: 135, done.
remote: Total 135 (delta 0), reused 0 (delta 0), pack-reused 135
Receiving objects: 100% (135/135), 8.96 MiB | 4.46 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import regex as re

In [ ]:
MAX_LEN = 50
BATCH_SIZE = 32

In [ ]:
# Ham xu ly chuỗi sai dấu
# ví dụ:  'Nhe ̣ nha ̀ ng sâu lă ́ ng nhưng cu ̃ ng triê ́ t ly ́ va ̀ tinh tê ́ .'
def chuan_hoa_dau(s):
    dau_nang_sai = ' ̣ '
    dau_nang_dung = dau_nang_sai[1]
    dau_huyen_sai = ' ̀ '
    dau_huyen_dung = dau_huyen_sai[1]
    dau_sac_sai = ' ́ '
    dau_sac_dung = dau_sac_sai[1]
    dau_nga_sai = ' ̃ '
    dau_nga_dung = dau_nga_sai[1]
    dau_hoi_sai = ' ̉ '
    dau_hoi_dung = dau_hoi_sai[1]
    dau_cau = {dau_nang_sai : dau_nang_dung,
           dau_huyen_sai : dau_huyen_dung,
           dau_nga_sai : dau_nga_dung,
           dau_sac_sai : dau_sac_dung,
           dau_hoi_sai : dau_hoi_dung}
    nguyen_am = 'aáàạảãăắằằẳẵâấầậẩẫeéèẻèẽêếềểễệiỉíìĩịoóòỏõọôốồộổỗơớờởỡợuúùụủũưứừựửữyýỳỷỹỵ'

    for k, v in dau_cau.items():
        while(s.find(k) >= 0):
            pos = s.find(k)
            s1 = s[pos+3:]
            # s2 là những ký tự phía sau dấu câu sai
            s2 = s1[:s1.find(' ')]
            if len(s2) == 2:
                #phía sau là môt từ
                if (nguyen_am.find(s2[0])>=0) | (nguyen_am.find(s2[1])>=0):
                    s=s.replace(k,v + ' ',1)
                #phia sau k phai 1 tu
                else: 
                    s=s.replace(k,v,1)
            #phia sau la mot tu
            elif len(s2) > 2:
                s=s.replace(k,v + ' ',1)
            #phia sau chi co 1 ky tu va no la 1 tu dac biet ngan
            elif 'áàạéèẻèêếềiíịoóòỏôốồơớờởợuúùụủũưứừựyýỳỵ'.find(s)>0:
                s=s.replace(k,v + ' ',1)
            else:
                s=s.replace(k,v,1)
    return s

In [ ]:
!gdown --id 1TzXkIicD3MsI6ciKnsDfSX4vFmYqd3LV
#download teencode.txt

Downloading...
From: https://drive.google.com/uc?id=1TzXkIicD3MsI6ciKnsDfSX4vFmYqd3LV
To: /content/teencode.txt
100% 5.66k/5.66k [00:00<00:00, 4.66MB/s]


In [ ]:
f = open('/content/teencode.txt','r', encoding='UTF-8')
teencode_dict = {}
for aline in f.readlines():
    aline = aline.split()
    teencode_dict[aline[0]] = ' '.join(aline[1:])
# teencode_dict

In [ ]:
def load_data(folder_id):
    train_data = []
    train_labels = []    
    test_data = []
    test_labels = []
    for label in range(3):

        # read train data
        path = '/content/MultiChannel/data/VS/Data_Token/Fold_{}/train_nhan_{}.txt'.format(str(folder_id),str(label))
        f = open(path, "r",encoding='utf-8-sig')
        regex = re.compile('[-:()\",.!?=`~/+&$%^*#]')
        text = []
        for aline in f.readlines():
            aline = chuan_hoa_dau(aline)
            aline = aline.lower()
            aline = regex.sub('', aline).strip()
            aline = re.sub(' +',' ',aline)
            for k,v in teencode_dict.items():
                aline.replace(k,v)
            if len(aline.split())==0:
                continue
            text.append(aline)
        train_labels += [label]*len(text)
        print('label: {}, len: {}'.format(label, len(text)))
        train_data +=text
        f.close()

        # read test data
        path = '/content/MultiChannel/data/VS/Data_Token/Fold_{}/test_nhan_{}.txt'.format(str(folder_id),str(label))
        f = open(path, "r",encoding='utf-8-sig')
        regex = re.compile('[-:()\",.!?=`~/+&$%^*#]')
        text = []
        for aline in f.readlines():
            aline = chuan_hoa_dau(aline)
            aline = aline.lower()
            aline = regex.sub('', aline).strip()
            aline = re.sub(' +',' ',aline)
            for k,v in teencode_dict.items():
                aline.replace(k,v)
            if len(aline.split())==0:
                continue
            text.append(aline)
        test_labels += [label]*len(text)
        test_data += text
        f.close()
    return train_data, train_labels, test_data, test_labels

In [ ]:
folder_id = 1
train_data, train_labels, test_data, test_labels = load_data(folder_id=folder_id)

label: 0, len: 9446
label: 1, len: 7304
label: 2, len: 9292


In [ ]:
print(len(train_data))
print(len(train_labels))
print(len(test_data))
print(len(test_labels))

26042
26042
8917
8917


In [1]:
!pip install transformers
import transformers
from transformers import AutoModel, AutoTokenizer, BertModel
import torch
phobert_model = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

In [ ]:
def preprocessing_for_phobert(data):
    input_ids = []
    attention_masks = []
    # token_type_ids = []
    for sentence in data:
        encoded = tokenizer.encode_plus(
            text = sentence,
            add_special_tokens = True,
            max_length = MAX_LEN,
            truncation  = True,
            pad_to_max_length = True,
            return_token_type_ids = True,
            return_attention_mask = True,
            # return_tensors = 'pt'
        )
        input_ids.append(encoded.get('input_ids'))
        attention_masks.append(encoded.get('attention_mask'))
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [ ]:
train_inputs, train_masks = preprocessing_for_phobert(train_data)
test_inputs, test_masks = preprocessing_for_phobert(test_data)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
train_labels = torch.tensor(train_labels)
train_dataset = torch.utils.data.TensorDataset(train_inputs, train_masks, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,
                                           pin_memory =True, shuffle=True)
test_labels = torch.tensor(test_labels)
test_dataset = torch.utils.data.TensorDataset(test_inputs, test_masks, test_labels)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE,
                                           pin_memory =True, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from sklearn.SVM import SVC
class MyClassifier(nn.Module):
    def __init__(self, num_labels , freeze_phobert = False):
        super(MyClassifier, self).__init__()
        self.num_labels = num_labels
        # self.phobert = phobert_model
        self.fc_1 = nn.Linear(768, 64)
        self.fc_2 = nn.Linear(64,num_labels)
        self.relu = nn.ReLU(inplace=True)

        # if freeze_phobert:
        #     for param in self.phobert.parameters():
        #         param.requires_grad = False
        ###
    def forward(self,x):
        # phobert_outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask)
        # cls = phobert_outputs[0][:, 0, :].squeeze()
        # print(cls.shape)
        x = self.relu(self.fc_1(x))
        x = self.fc_2(x)
        return x

In [ ]:

def phobert_training_loop(n_epochs, optimizer, model, loss_fn, train_loader, test_loader):
    highest_acc = 0.6361
    model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/sentiment_analysis/folder{}_PhoBert.pth".format(folder_id)))
    for epoch in range(1,n_epochs+1):
        model.train()
        loss_train = 0.0
        for input_ids,attn_mask ,labels in train_loader:

            input_ids = input_ids.type(torch.LongTensor)
            attn_mask = attn_mask.type(torch.LongTensor)
            labels = labels.type(torch.LongTensor)

            phobert_outputs = phobert_model(input_ids=input_ids.to(device), attention_mask=attn_mask.to(device))
            cls = phobert_outputs[0][:, 0, :].squeeze()
            outputs = model(cls)
            # print(outputs[:5])
            optimizer.zero_grad()
            loss = loss_fn(outputs, labels.to(device))
            
            loss.backward()
            optimizer.step()
            loss_train +=loss.item()
            # break
        correct = 0
        model.eval()
        with torch.no_grad():
            for data in test_loader:
                input_id ,att_mask ,label = data
                input_id = input_id.type(torch.LongTensor)
                att_mask = att_mask.type(torch.LongTensor)
                label = label.type(torch.LongTensor)

                phobert_output = phobert_model(input_ids=input_id.to(device), attention_mask=att_mask.to(device))
                cls = phobert_output[0][:, 0, :].squeeze()
                output = model(cls)
                predicted = torch.argmax(output,1)
                # print(predicted[:5])
                c = (predicted == label.to(device))#.squeeze()
                correct += c.sum()
                # break
        if epoch == 1 or epoch % 1 == 0:
            print('Epoch {}, Training loss {}, Val accuracy {}'.format(
                epoch,
                loss_train / len(train_loader),
                correct / len(test_labels)))
        if correct/len(test_labels) > highest_acc:
            highest_acc = correct/len(test_labels)
            print(highest_acc)
            torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/sentiment_analysis/folder{}_PhoBert.pth".format(folder_id))
            # break
        # break
    print(highest_acc)




In [ ]:
def get_default_device():
    """Picking GPU if available or else CPU"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        return torch.device('cuda')
    else:
        return torch.device('cpu')
device = get_default_device()

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)
class DevicedataLoader():
    def __init__(self,dl,device):
        self.dl = dl
        self.device = device
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    def __len__(self):
        return len(self.dl)
print(device)

cuda


In [ ]:
train_loader = DevicedataLoader(train_loader,device)
test_loader = DevicedataLoader(test_loader,device)

In [ ]:
num_labels = 3
learning_rate = 1e-4
phobert_model.to(device)
phobert = MyClassifier(num_labels = num_labels,freeze_phobert=True)
phobert.to(device)
optimizer = optim.Adam(phobert.parameters(), lr=learning_rate)
loss_fn = F.cross_entropy

In [2]:
phobert_training_loop(
    n_epochs = 10,
    optimizer = optimizer,
    model = phobert,
    loss_fn = loss_fn,
    train_loader = train_loader,
    test_loader = test_loader
)